# Initialization
For data in CNTC lab only

In [1]:
from safedigital import temperature as TR  # 从safedigital包中导入temperature库
import pandas as pd
import os
os.chdir('../')
cur_dir = os.getcwd()
print(cur_dir)

def data_clean(test_date,device_id):
    sample_time = 1  # 降采样系数列表
    sample_str = '10s'  # 采样时间列表
    test_folder = test_date + device_id  # 试验文件夹
    folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
    folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
    file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
    file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
    path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
    path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
    raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
    raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
    print(raw_sen)
    # print(raw_coup1)
    syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳
    chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, 1)  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, 1)  # 热电偶1数据降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

    col_sen = list(syn_sen.columns)
    col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
    col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

def data_clean_direct_temp(test_date,device_id):
    
    sample_str = '10s'  # 采样时间列表
    test_folder = test_date + device_id  # Test folder
    folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # raw data folder
    folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # folder for washed data
    file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # file name of MDC4 recorded data
    file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # file name of data from thermal couples
    file_name_direct = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TD.csv'  # file name of data from direct temp sensor recorded by supplier's logger
    path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data from MDC4-M
    path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
    path_direct = folder_data_raw + '\\' + file_name_direct  # file location of direct temp sensor data 
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_direct_%s.csv' % (test_date, sample_str)  # location for washed data
    raw_sen = TR.DataClean.read_logger_data(path_sensor)  # read MDC4-M data
    raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # read thermal couples data
    # print(raw_coup1.columns)
    for i in range(4, raw_coup1.shape[1] - 1):  # channel 1-59 data shall add channel 60 since they are deviation against channel 60
        raw_coup1.iloc[:, i] = raw_coup1.iloc[:, i].values + raw_coup1.iloc[:, -2].values
    raw_direct = TR.DataClean.read_gaoli_logger_data(path_direct)  # read thermal couples data
    # print(raw_sen)
    print(raw_coup1.columns)
    syn_sen, syn_coup1, syn_direct = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1, raw_direct)  # synchronize all data groups 
    # syn_sen = TR.DataClean.down_sample(syn_sen, 1)  # down-sample
    # syn_coup1 = TR.DataClean.down_sample(syn_coup1, 1)  # down-sample
    col_sen = ['datetime_sen'] + list(syn_sen.columns)
    col_data_sen = [syn_sen.iloc[:, i].values for i in range(syn_sen.shape[1])]

    col_coup1 = ['datetime_coup1'] + ['ch%d' % i for i in range(1, 61)]
    col_data_coup1 = [syn_coup1.iloc[:, i].values for i in list(range(4, syn_coup1.shape[1]))]

    col_direct = ['datetime_direct'] + ['GL1A', 'GL1B', 'GL1C']
    col_data_direct = [syn_direct.loc[:, i].values for i in ['GL1A', 'GL1B', 'GL1C']]

    col_all = col_sen + col_coup1 + col_direct
    col_data_all = col_data_sen + col_data_coup1 + col_data_direct
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    print(data_clean_df.columns)
    print(data_clean_df)
    data_clean_df.to_csv(path_data_clean)

c:\Users\cnbofan1\ABB\Safe Digital in CN - Documents - Documents\06_Test and Analytics


# 20211019

In [ ]:
test_date = '20211019'  # 试验日期
device_id = '_TR SR40.5kV cable skin vert 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211029

In [ ]:
test_date = '20211029'  # 试验日期
device_id = '_DTR&TR_SR12kV 100A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211102

In [ ]:
test_date = '20211102'  # 试验日期
device_id = '_DTR&TR_3in1_SR12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211103

In [ ]:
test_date = '20211103'  # 试验日期
device_id = '_DTR&TR_3in1_SR12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211104

In [ ]:
test_date = '20211104'  # 试验日期
device_id = '_DTR&TR_3in1_SR12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211105

In [ ]:
test_date = '20211105'  # 试验日期
device_id = '_DTR&TR_SR12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211106

In [ ]:
test_date = '20211106'  # 试验日期
device_id = '_DTR&TR_SR12kV Current Variant'
sample_time_list = [90]  # 降采样系数列表
sample_str_list = ['15min']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211107

In [ ]:
test_date = '20211107'  # 试验日期
device_id = '_DTR&TR_SR12kV Current Variant'
sample_time_list = [90]  # 降采样系数列表
sample_str_list = ['15min']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211118


In [ ]:
test_date = '20211118'  # 试验日期
device_id = '_DTR&TR_SR12kV Current Drop'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 19))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211123


In [ ]:
test_date = '20211123'  # 试验日期
device_id = '_DTR&TR_SR12kV Fault Simulation'
sample_time_list = [30]  # 降采样系数列表
sample_str_list = ['5m']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
               'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211124


In [ ]:
test_date = '20211124'  # 试验日期
device_id = '_DTR&TR_SR12kV Fault Simulation'
sample_time_list = [30]  # 降采样系数列表
sample_str_list = ['5min']  # 采样时间列表
sample_time_dict = dict(zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
    syn_sen = TR.DataClean.down_sample(
        syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
    syn_coup1 = TR.DataClean.down_sample(
        syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
    # syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

    for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
        syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values
    # for i in range(4,chn_num_tc1+3):
    # 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
    col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
               't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
               'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
    col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
    col_tc1 = ['ch%d' % i for i in range(
        1, 61)] + ['data_time_coup1', 'ind_coup1']
    col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
    # col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
    # col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
    col_all = col_sen + col_tc1
    col_data_all = col_data_sen + col_data_tc1
    data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
    data_clean_df.to_csv(path_data_clean)

# 20211208


In [ ]:
test_date = '20211208'  # 试验日期
device_id = '_DTR&TR_SA12kV 300A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211209
Constant current 630A till steady-state


In [ ]:
test_date = '20211209'  # 试验日期
device_id = '_DTR&TR_SA12kV 630A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211210
Constant current 100A till steady-state


In [ ]:
test_date = '20211210'  # 试验日期
device_id = '_DTR&TR_SA12kV 100A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211216
Variant current 100-300-500-630-400-200


In [ ]:
test_date = '20211216'  # 试验日期
device_id = '_DTR&TR_SA12kV Current Variant'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211217
Variant current 100-300-500-630-400-200


In [ ]:
test_date = '20211217'  # 试验日期
device_id = '_DTR&TR_SA12kV Current Variant'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211218
Constant current 500A till steady-state


In [ ]:
test_date = '20211218'  # 试验日期
device_id = '_DTR&TR_SA12kV 500A'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211219
Variant current 100-300-500-630-400-200


In [ ]:
test_date = '20211219'  # 试验日期
device_id = '_DTR&TR_SA12kV Current Variant'
sample_time_list = [90]  # 降采样系数列表
sample_str_list = ['15min']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['date_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211220
Variant current with fault 100-300-500-630-400-200


In [ ]:
test_date = '20211220'  # 试验日期
device_id = '_DTR&TR_SA12kV Fault Simulation'
sample_time_list = [30]  # 降采样系数列表
sample_str_list = ['5min']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211221
630A constant current with fault 


In [ ]:
test_date = '20211221'  # 试验日期
device_id = '_DTR&TR_SA12kV Fault Simulation'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)


# 20211222
630A-100A current drop without fault 


In [ ]:
test_date = '20211222'  # 试验日期
device_id = '_DTR&TR_SA12kV Fault Simulation'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_sensor_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_data_group(
    raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = ['t_sen1', 't_sen2', 't_sen3', 't_sen4', 't_sen5', 't_sen6', 't_sen7', 't_sen8', 't_sen9',
           't_amb_sen', 'hum_amb_sen', 't_mano', 'p_mano', 'p20_mano', 'P20_avg_mano', 'amp_phA',
           'amp_phB', 'amp_phC', 'date_time_sen', 'ind_sen']
col_data_sen = [syn_sen.iloc[:, i].values for i in list(range(2, 22))]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['data_time_coup1', 'ind_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
# col_tc2 = ['ch%d' % i for i in range(61,61+chn_num_tc2)]
# col_data_tc2 = [syn_coup2.iloc[:,i].values for i in list(range(4,4+chn_num_tc2))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

# 20220222

In [ ]:
test_date = '20220222'  # 试验日期
device_id = '_TR_3in1_SR12kV Cable Skin'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220224

In [ ]:
test_date = '20220224'  # 试验日期
device_id = '_TR_3in1_SR12kV Cable Skin'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220225

In [ ]:
test_date = '20220225'  # 试验日期
device_id = '_TR_3in1_SR12kV Cable Skin'
sample_time_list = [1]  # 降采样系数列表
sample_str_list = ['10s']  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder +
'\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder +
'\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] +
'-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' +
file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' +
file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据
for sample_ind, sample_str in enumerate(sample_str_list):
    path_data_clean = folder_data_clean + '\\' +
    '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样
# syn_coup2 = dc.DataClean.down_sample(syn_coup2,sample_time_dict[sample_str]) # 降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values +
    syn_coup1.iloc[:, 63].values
# for i in range(4,chn_num_tc1+3):
# 		syn_coup2.iloc[:,i] = syn_coup2.iloc[:,i].values + syn_coup2.iloc[:,chn_num_tc2+3].values
col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(
    1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220226

In [ ]:
test_date = '20220226'  # 试验日期
device_id = '_TR_3in1_SR12kV Fault'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
sample_time_dict = dict(
    zip(sample_str_list, sample_time_list))  # 建立采样间隔与降采样比例的字典
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, sample_time_dict[sample_str])  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, sample_time_dict[sample_str])  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220228

In [ ]:
test_date = '20220228'  # 试验日期
device_id = '_TR_3in1_SR12kV Fault'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220302

In [ ]:
test_date = '20220302'  # 试验日期
device_id = '_TR_3in1_SR12kV_400mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220303

In [ ]:
test_date = '20220303'  # 试验日期
device_id = '_TR_3in1_SR12kV_400mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220304

In [ ]:
test_date = '20220304'  # 试验日期
device_id = '_TR_3in1_SR12kV_400mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220308

In [ ]:
test_date = '20220308'  # 试验日期
device_id = '_TR_3in1_SR12kV_400mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220309

In [ ]:
test_date = '20220309'  # 试验日期
device_id = '_TR_3in1_SR12kV_300mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220316

In [ ]:
test_date = '20220316'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220317

In [ ]:
test_date = '20220317'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220318

In [ ]:
test_date = '20220318'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
sample_time = 1  # 降采样系数列表
sample_str = '10s'  # 采样时间列表
test_folder = test_date + device_id  # 试验文件夹
folder_data_raw = cur_dir + '\\' + test_folder + '\\' + '0_Data original'  # 原始数据文件夹
folder_data_clean = cur_dir + '\\' + test_folder + '\\' + '1_Data formatted'  # 清理完数据文件夹
file_name_sen = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TP.csv'  # 传感器数据文件名
file_name_coup1 = test_date[0:4] + '-' + test_date[4:6] + '-' + test_date[6:8] + '_TC1.csv'  # 热电偶数据文件名'
path_sensor = folder_data_raw + '\\' + file_name_sen  # file location of sensor data
path_coup1 = folder_data_raw + '\\' + file_name_coup1  # file location of coupler data
raw_sen = TR.DataClean.read_logger_data(path_sensor)  # 读取数据
raw_coup1 = TR.DataClean.read_coupler_data(path_coup1)  # 读取数据
syn_sen, syn_coup1 = TR.DataClean.synch_logger_couple(raw_sen, raw_coup1)  # 同步三组数据时间戳

chn_num_tc1 = syn_coup1.shape[1] - 6  # 热电偶1的列数：前四列为无效数据，后两列为时间戳和索引

# 生成不同采样间隔的清洗数据

path_data_clean = folder_data_clean + '\\' + '%s_data_clean_%s.csv' % (test_date, sample_str)  # 清洗完文件位置
syn_sen = TR.DataClean.down_sample(
    syn_sen, 1)  # 传感器数据降采样
syn_coup1 = TR.DataClean.down_sample(
    syn_coup1, 1)  # 热电偶1数据降采样

for i in range(4, chn_num_tc1 + 3):  # 遍历所有热电偶通道的数据，实际温度为本通道温度+通道60的温度
    syn_coup1.iloc[:, i] = syn_coup1.iloc[:, i].values + syn_coup1.iloc[:, 63].values

col_sen = list(syn_sen.columns)
col_data_sen = [syn_sen.iloc[:, i].values for i in range(26)]
col_tc1 = ['ch%d' % i for i in range(1, 61)] + ['datetime_coup1', 'idx_coup1']
col_data_tc1 = [syn_coup1.iloc[:, i].values for i in list(range(4, 66))]
col_all = col_sen + col_tc1
col_data_all = col_data_sen + col_data_tc1
data_clean_df = pd.DataFrame(dict(zip(col_all, col_data_all)))
data_clean_df.to_csv(path_data_clean)

#20220322
100-300-500-630-400-200A

In [ ]:
test_date = '20220322'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
data_clean(test_date,device_id)

#20220323
630-700-630A till stable

In [ ]:
test_date = '20220323'  # 试验日期
device_id = '_TR_3in1_SR40.5kV_300mm2 Cable Skin'
data_clean(test_date,device_id)

# 20220714-2022????
## 20220714

In [ ]:
test_date = '20220714'  # 试验日期
device_id = '_TR_SA12kV_SGCC_Top_Extension_No_Cover_630A'
data_clean(test_date,device_id)

## 20220719

In [ ]:
test_date = '20220719'  # 试验日期
device_id = '_TR_SA12kV_SGCC_Top_Extension_With_Cover_630A'
data_clean(test_date,device_id)

## 20220720

In [ ]:
test_date = '20220720'  # 试验日期
device_id = '_TR_SA12kV_SGCC_Top_Extension_With_Cover_630A_700A_630A'
data_clean(test_date,device_id)

## 20220721

In [ ]:
test_date = '20220721'  # 试验日期
device_id = '_TR_SA12kV_SGCC_Top_Extension_With_Cover_700A'
data_clean(test_date,device_id)

## 20220725

In [ ]:
test_date = '20220725'  # 试验日期
device_id = '_TR_SA12kV_SGCC_Top_Extension_With_Cover_500A_630A_500A'
data_clean(test_date,device_id)

# 20220902 - 202209??
## 20220902

In [2]:
test_date = '20220902'  # test date 
device_id = '_TR_SA12kV_SGCC_DV_Cable_Bushing_630A' # test name
data_clean(test_date,device_id)

0       2022-09-02 10:37:25
1       2022-09-02 10:37:35
2       2022-09-02 10:37:45
3       2022-09-02 10:37:55
4       2022-09-02 10:38:05
               ...         
3339    2022-09-02 19:54:24
3340    2022-09-02 19:54:34
3341    2022-09-02 19:54:44
3342    2022-09-02 19:54:54
3343    2022-09-02 19:55:04
Name: Time, Length: 3344, dtype: object
logger started from 10:41:56
logger ended at 19:54:34
Index(['Mark', 'Date', 'Time', 'Second', 'C', 'C.1', 'C.2', 'C.3', 'C.4',
       'C.5', 'C.6', 'C.7', 'C.8', 'C.9', 'C.10', 'C.11', 'C.12', 'C.13',
       'C.14', 'C.15', 'C.16', 'C.17', 'C.18', 'C.19', 'C.20', 'C.21', 'C.22',
       'C.23', 'C.24', 'C.25', 'C.26', 'C.27', 'C.28', 'C.29', 'C.30', 'C.31',
       'C.32', 'C.33', 'C.34', 'C.35', 'C.36', 'C.37', 'C.38', 'C.39', 'C.40',
       'C.41', 'C.42', 'C.43', 'C.44', 'C.45', 'C.46', 'C.47', 'C.48', 'C.49',
       'C.50', 'C.51', 'C.52', 'C.53', 'C.54', 'C.55', 'C.56', 'C.57', 'C.58',
       'C.59'],
      dtype='object')
                d

### data washing for test with direct TR data

In [2]:
test_date = '20220902'  # test date 
device_id = '_TR_SA12kV_SGCC_DV_Cable_Bushing_630A' # test name
data_clean_direct_temp(test_date,device_id)

0       2022-09-02 10:37:25
1       2022-09-02 10:37:35
2       2022-09-02 10:37:45
3       2022-09-02 10:37:55
4       2022-09-02 10:38:05
               ...         
3339    2022-09-02 19:54:24
3340    2022-09-02 19:54:34
3341    2022-09-02 19:54:44
3342    2022-09-02 19:54:54
3343    2022-09-02 19:55:04
Name: Time, Length: 3344, dtype: object
logger started from 10:41:56
logger ended at 19:54:34
Index(['Mark', 'Date', 'Time', 'Second', 'C', 'C.1', 'C.2', 'C.3', 'C.4',
       'C.5', 'C.6', 'C.7', 'C.8', 'C.9', 'C.10', 'C.11', 'C.12', 'C.13',
       'C.14', 'C.15', 'C.16', 'C.17', 'C.18', 'C.19', 'C.20', 'C.21', 'C.22',
       'C.23', 'C.24', 'C.25', 'C.26', 'C.27', 'C.28', 'C.29', 'C.30', 'C.31',
       'C.32', 'C.33', 'C.34', 'C.35', 'C.36', 'C.37', 'C.38', 'C.39', 'C.40',
       'C.41', 'C.42', 'C.43', 'C.44', 'C.45', 'C.46', 'C.47', 'C.48', 'C.49',
       'C.50', 'C.51', 'C.52', 'C.53', 'C.54', 'C.55', 'C.56', 'C.57', 'C.58',
       'C.59'],
      dtype='object')
Index(['Time', 'S

KeyError: 1

## 20220905

In [6]:
test_date = '20220905'  # 试验日期
device_id = '_TR_SA12kV_SGCC_DV_Cable_Bushing_700A'
data_clean(test_date,device_id)

0       2022-09-05 09:02:23
1       2022-09-05 09:02:33
2       2022-09-05 09:02:43
3       2022-09-05 09:02:53
4       2022-09-05 09:03:03
               ...         
2807    2022-09-05 16:50:36
2808    2022-09-05 16:50:46
2809    2022-09-05 16:50:56
2810    2022-09-05 16:51:06
2811    2022-09-05 16:51:16
Name: Time, Length: 2812, dtype: object
logger started from 09:11:43
test data are not fully recorded
Index(['Mark', 'Date', 'Time', 'Second', 'C', 'C.1', 'C.2', 'C.3', 'C.4',
       'C.5', 'C.6', 'C.7', 'C.8', 'C.9', 'C.10', 'C.11', 'C.12', 'C.13',
       'C.14', 'C.15', 'C.16', 'C.17', 'C.18', 'C.19', 'C.20', 'C.21', 'C.22',
       'C.23', 'C.24', 'C.25', 'C.26', 'C.27', 'C.28', 'C.29', 'C.30', 'C.31',
       'C.32', 'C.33', 'C.34', 'C.35', 'C.36', 'C.37', 'C.38', 'C.39', 'C.40',
       'C.41', 'C.42', 'C.43', 'C.44', 'C.45', 'C.46', 'C.47', 'C.48', 'C.49',
       'C.50', 'C.51', 'C.52', 'C.53', 'C.54', 'C.55', 'C.56', 'C.57', 'C.58',
       'C.59'],
      dtype='object')
         

## 20220906

In [9]:
test_date = '20220906'  # 试验日期
device_id = '_TR_SA12kV_SGCC_DV_Cable_Bushing_500A_630A_500A'
data_clean(test_date,device_id)

0       2022-09-06 09:23:33
1       2022-09-06 09:23:44
2       2022-09-06 09:23:54
3       2022-09-06 09:24:03
4       2022-09-06 09:24:14
               ...         
3253    2022-09-06 18:26:11
3254    2022-09-06 18:26:21
3255    2022-09-06 18:26:31
3256    2022-09-06 18:26:41
3257    2022-09-06 18:26:51
Name: Time, Length: 3258, dtype: object
logger started from 09:24:03
logger ended at 18:25:01
Index(['Mark', 'Date', 'Time', 'Second', 'C', 'C.1', 'C.2', 'C.3', 'C.4',
       'C.5', 'C.6', 'C.7', 'C.8', 'C.9', 'C.10', 'C.11', 'C.12', 'C.13',
       'C.14', 'C.15', 'C.16', 'C.17', 'C.18', 'C.19', 'C.20', 'C.21', 'C.22',
       'C.23', 'C.24', 'C.25', 'C.26', 'C.27', 'C.28', 'C.29', 'C.30', 'C.31',
       'C.32', 'C.33', 'C.34', 'C.35', 'C.36', 'C.37', 'C.38', 'C.39', 'C.40',
       'C.41', 'C.42', 'C.43', 'C.44', 'C.45', 'C.46', 'C.47', 'C.48', 'C.49',
       'C.50', 'C.51', 'C.52', 'C.53', 'C.54', 'C.55', 'C.56', 'C.57', 'C.58',
       'C.59'],
      dtype='object')
                d

## 20220907

In [10]:
test_date = '20220907'  # 试验日期
device_id = '_TR_SA12kV_SGCC_DV_Cable_Bushing_630A_700A_630A'
data_clean(test_date,device_id)

0       2022-09-07 09:06:16
1       2022-09-07 09:06:26
2       2022-09-07 09:06:36
3       2022-09-07 09:06:46
4       2022-09-07 09:06:56
               ...         
2710    2022-09-07 16:38:19
2711    2022-09-07 16:38:29
2712    2022-09-07 16:38:39
2713    2022-09-07 16:38:49
2714    2022-09-07 16:38:59
Name: Time, Length: 2715, dtype: object
logger started from 09:06:16
logger ended at 16:38:59
Index(['Mark', 'Date', 'Time', 'Second', 'C', 'C.1', 'C.2', 'C.3', 'C.4',
       'C.5', 'C.6', 'C.7', 'C.8', 'C.9', 'C.10', 'C.11', 'C.12', 'C.13',
       'C.14', 'C.15', 'C.16', 'C.17', 'C.18', 'C.19', 'C.20', 'C.21', 'C.22',
       'C.23', 'C.24', 'C.25', 'C.26', 'C.27', 'C.28', 'C.29', 'C.30', 'C.31',
       'C.32', 'C.33', 'C.34', 'C.35', 'C.36', 'C.37', 'C.38', 'C.39', 'C.40',
       'C.41', 'C.42', 'C.43', 'C.44', 'C.45', 'C.46', 'C.47', 'C.48', 'C.49',
       'C.50', 'C.51', 'C.52', 'C.53', 'C.54', 'C.55', 'C.56', 'C.57', 'C.58',
       'C.59'],
      dtype='object')
                d

## 20220908

In [11]:
test_date = '20220908'  # 试验日期
device_id = '_TR_SA12kV_SGCC_DV_Cable_Bushing_630A_Fault'
data_clean(test_date,device_id)

0       2022-09-08 08:49:25
1       2022-09-08 08:49:35
2       2022-09-08 08:49:45
3       2022-09-08 08:49:55
4       2022-09-08 08:50:05
               ...         
2848    2022-09-08 16:44:29
2849    2022-09-08 16:44:39
2850    2022-09-08 16:44:49
2851    2022-09-08 16:44:59
2852    2022-09-08 16:45:09
Name: Time, Length: 2853, dtype: object
logger started from 08:50:15
logger ended at 16:24:38
Index(['Mark', 'Date', 'Time', 'Second', 'C', 'C.1', 'C.2', 'C.3', 'C.4',
       'C.5', 'C.6', 'C.7', 'C.8', 'C.9', 'C.10', 'C.11', 'C.12', 'C.13',
       'C.14', 'C.15', 'C.16', 'C.17', 'C.18', 'C.19', 'C.20', 'C.21', 'C.22',
       'C.23', 'C.24', 'C.25', 'C.26', 'C.27', 'C.28', 'C.29', 'C.30', 'C.31',
       'C.32', 'C.33', 'C.34', 'C.35', 'C.36', 'C.37', 'C.38', 'C.39', 'C.40',
       'C.41', 'C.42', 'C.43', 'C.44', 'C.45', 'C.46', 'C.47', 'C.48', 'C.49',
       'C.50', 'C.51', 'C.52', 'C.53', 'C.54', 'C.55', 'C.56', 'C.57', 'C.58',
       'C.59'],
      dtype='object')
                d